In [2]:
import pandas as pd
df = pd.read_csv(
)
df

,X1,X2,X3,X4,Y
0,0.334585,-0.219956,-0.004864,-0.259753,11.471628
1,1.216243,0.725734,-0.150105,0.442589,23.642019
2,1.171834,1.489064,0.533833,1.229934,59.935247
3,1.851015,1.725315,1.272595,1.370409,137.028250
4,2.341974,1.611095,2.060412,1.855743,207.806090
5,2.642522,2.218922,2.266301,3.204849,412.123190
6,3.327416,3.076885,2.996936,3.352975,865.194490
7,3.170240,3.566735,3.872283,3.224383,1181.472700
8,4.114775,4.119899,3.719884,4.115981,1789.786000
9,4.542168,4.467874,4.905525,4.328305,2573.796900


In [3]:
Z1 = (df['X1']).rename('Z1')
Z2 = ((df['X2']**2)*df['X1']).rename('Z2')
Z3 = ((df['X2']**2)*df['X3']).rename('Z3')
Z4 = (df['X1']*df['X3']*df['X4']).rename('Z4')
augdf = pd.concat([Z1, Z2, Z3, Z4, df['Y']], axis=1)
augdf.to_csv('augdata.csv', header=True, index=False)
augdf

,Z1,Z2,Z3,Z4,Y
0,0.334585,0.016187,-0.000235,0.000423,11.471628
1,1.216243,0.640583,-0.079059,-0.080801,23.642019
2,1.171834,2.598322,1.183675,0.769402,59.935247
3,1.851015,5.509938,3.788148,3.228125,137.028250
4,2.341974,6.078887,5.348057,8.954760,207.806090
5,2.642522,13.010767,11.158397,19.193042,412.123190
6,3.327416,31.501397,28.372669,33.436054,865.194490
7,3.170240,40.330521,49.261627,39.582737,1181.472700
8,4.114775,69.842424,63.139715,63.001195,1789.786000
9,4.542168,90.670299,97.923596,96.442089,2573.796900


In [6]:
import numpy as np
from scipy.stats import f, t
class MultipleRegression():
    def __init__(self, augdf, alpha):
        self.X = augdf.to_numpy()[:, 0:4]
        self.X = np.insert(self.X, 0, np.ones((20,)), axis=1)
        self.y = df.to_numpy()[:, 4]
        self.y = self.y.reshape(-1, 1)
        self.alpha = alpha
        self.n = 20
        self.p = 5
        self.regDOF = self.p - 1
        self.errDOF = self.n - self.p
        self.totDOF = self.n - 1
        self.Bhat = np.dot(np.linalg.inv(np.dot(self.X.T, self.X)), np.dot(self.X.T, self.y))
        self.yhat = np.dot(self.X, self.Bhat)
        self.SSE = np.sum((self.y - self.yhat)**2)
        self.SST = np.sum((self.y - np.mean(self.y))**2)
        self.SSR = self.SST - self.SSE
        self.mSSR = self.SSR/self.regDOF
        self.mSSE = self.SSE/self.errDOF
        self.RMSE = np.sqrt(self.mSSE)
        self.Fstat = self.mSSR/self.mSSE
        self.p_Ftest = 2*(1-f.cdf(self.Fstat, self.regDOF, self.errDOF))
        self.Rsq = self.SSR/self.SST
        self.Rsq_adj = 1 - (self.n - 1)*(1 - self.Rsq)/(self.n - self.p)
        self.H0_Ftest = None
        self.CovBhat = [[]]
        self.SE = []
        self.Tstats = []
        self.pvalues = []
        self.sigs = []
        self.errors_at_alpha = []

    def ftest(self):
        if f.ppf(self.alpha/2, self.regDOF, self.errDOF) <= self.Fstat <= f.ppf(1-self.alpha/2, self.regDOF, self.errDOF):
            self.H0_Ftest = 1
            # print('Null Hypothesis is true')
            
        else:
            self.H0_Ftest = 0
            # print('Null Hypothesis Rejected')
        return self.H0_Ftest
    def estimate(self):
        if self.ftest() == 0:
            self.CovBhat = np.linalg.inv(np.dot(self.X.T, self.X))*self.mSSE
            for i in range(0,5):
                self.SE.append(np.sqrt(model.CovBhat[i][i]))
                self.Tstats.append(self.Bhat[i][0]/self.SE[i])
                self.pvalues.append(2*(1 - t.cdf(self.Tstats[i], self.errDOF)))
                self.errors_at_alpha.append(self.SE[i]*abs(t.ppf(self.alpha/2, self.errDOF)))

    def summary(self):
        self.estimate()
        self.sigs = ['Significant' if self.pvalues[i] < self.alpha else 'Not Significant' for i in range(0,len(self.pvalues))]
        results = pd.concat([pd.Series(np.reshape(model.Bhat, (self.p,)), name = 'Coefficients'), pd.Series(self.SE, name = 'SEs'), pd.Series(self.sigs, name = 'Significance'), pd.Series(self.errors_at_alpha, name = f'Errors at {round(self.alpha*100, 2)}% ')], axis=1)
        results.index = ['Intercept', 'Z1', 'Z2', 'Z3', 'Z4']
        return results

    def anova(self):
        C1 = pd.Series(np.array([self.regDOF, self.errDOF, self.totDOF]), name = 'DOF')
        C2 = pd.Series(np.array([self.SSR, self.SSE, self.SST]), name = 'Sum Sqs')
        C3 = pd.Series(np.array([self.mSSR, self.mSSE, '-']), name = 'Mean Sum Sqs')
        C4 = pd.Series(np.array([self.Fstat, '-', '-']), name = 'F-value')
        C5 = pd.Series([self.p_Ftest, '-', '-'], name = 'p-value')
        anova_table = pd.concat([C1, C2, C3, C4, C5], axis=1)
        anova_table.index = ['Residual', 'Error', 'Total']
        return anova_table

In [7]:
model = MultipleRegression(augdf, alpha=0.0145)
summary = model.summary()
summary

,Coefficients,SEs,Significance,Errors at 1.45%
Intercept,9.005056,0.445411,Significant,1.230697
Z1,8.056825,0.153088,Significant,0.422993
Z2,9.625502,0.016189,Significant,0.044730
Z3,8.776626,0.016766,Significant,0.046325
Z4,8.257979,0.004411,Significant,0.012189


In [8]:
tbl = model.anova()
print(f"\n Rsq = {model.Rsq}    Rsq adjusted = {model.Rsq_adj}")
tbl


 Rsq = 0.9999999928356722    Rsq adjusted = 0.9999999909251848


,DOF,Sum Sqs,Mean Sum Sqs,F-value,p-value
Residual,4,9.978289e+08,249457222.89633015,523426631.53832847,2.22045e-16
Error,15,7.148773e+00,0.47658488862743964,-,-
Total,19,9.978289e+08,-,-,-
